In [8]:
"""Training"""
Labels = {}
Accuracies = []

for name,group in labeled_data_train.groupby('state'):
    print(name)
    label = labeled_data_train.groupby('state').get_group(name)['predicted'].value_counts(normalize=True)
    print(label)
    Labels[name] = label.idxmax()
    Accuracies.append(label.max())

augmented_reality
6    0.977778
2    0.020833
3    0.001389
Name: predicted, dtype: float64
game
1    0.987437
2    0.012563
Name: predicted, dtype: float64
idle
5    0.988224
2    0.010795
3    0.000981
Name: predicted, dtype: float64
mining
4    0.952253
3    0.034297
2    0.013450
Name: predicted, dtype: float64
stream
0    0.91709
2    0.08291
Name: predicted, dtype: float64


In [11]:
"""Generates Label Predictions"""


for prediction_window in prediction_windows:

    predicted_observations_list, observations_list = [], []
    predicted_stateseq_list = []
    test_labels_window_list = []
    log_likelihoods_list = []

    print(f"{prediction_window}-step prediction")

    for i in tqdm(range(prediction_start,len(labeled_data_test)+1)):
        
        seed_start_idx =  i - lookback
        seed_end_idx = seed_start_idx + lookback
        
        if seed_end_idx > len(labeled_data_test)-prediction_window:
            break
        
        predicted_observations, predicted_stateseq, log_likelihood = SM.HSMM_pred(model,seed_start_idx, seed_end_idx, prediction_window)
        
        predicted_observations_list.append(predicted_observations[lookback:].tolist())
        predicted_stateseq_list.append(predicted_stateseq[lookback:].tolist())
        log_likelihoods_list.append(round(log_likelihood,2))
        observations_list.append(df[i:i+prediction_window].values)

        test_labels_window_list.append(test_labels[i:i+prediction_window])  

    # store results
    predicted_stateseq_list = ['lookback']*lookback + predicted_stateseq_list
    log_likelihoods_list = ['lookback']*lookback + log_likelihoods_list


    if len(labeled_data_test) != len(predicted_stateseq_list):
        
        if prediction_window != 1:
            labeled_data_test = labeled_data_test[:-(prediction_window-1)]

    labeled_data_test[f'predicted states - {prediction_window} step'] = predicted_stateseq_list
    labeled_data_test[f'log_likelihood - {prediction_window} step'] = log_likelihoods_list

print("Done!")

1-step prediction


100%|█████████▉| 2285/2286 [01:07<00:00, 33.64it/s]


2-step prediction


100%|█████████▉| 2284/2286 [01:19<00:00, 28.65it/s]


5-step prediction


...

30-step prediction


 99%|█████████▊| 2228/2258 [01:29<00:01, 24.92it/s]


60-step prediction


 97%|█████████▋| 2169/2229 [01:36<00:02, 22.57it/s]

Done!


In [12]:
"""Testing"""
for name,group in labeled_data_test.groupby('state'):
    print(name)
    print(labeled_data_test[lookback:].groupby('state').get_group(name)[f'predicted states - {1} step'].value_counts(normalize=True))

augmented_reality
[6]    0.908840
[2]    0.056630
[4]    0.019337
[1]    0.004144
[5]    0.004144
[3]    0.004144
[0]    0.002762
Name: predicted states - 1 step, dtype: float64
game
[1]    0.899160
[2]    0.092437
[4]    0.008403
Name: predicted states - 1 step, dtype: float64
idle
[5]    0.900538
[2]    0.069892
[1]    0.026882
[3]    0.002688
Name: predicted states - 1 step, dtype: float64
mining
[4]    0.838889
[3]    0.061111
[2]    0.055556
[5]    0.027778
[6]    0.013889
[0]    0.002778
Name: predicted states - 1 step, dtype: float64
stream
[0]    0.897436
[2]    0.076923
[5]    0.008547
[1]    0.004274
[3]    0.004274
[4]    0.004274
[6]    0.004274
Name: predicted states - 1 step, dtype: float64


In [21]:
for step in prediction_windows:
    accuracy = []
    for index, row in labeled_data_test.iterrows():
        accuracy.append(accuracy_score(row[f'label - {step} step'], row[f'predicted states - {step} step']))

    print(f"{step} step prediction accuracy: {int(round(mean(accuracy),2)*100)}%")

1 step prediction accuracy: 88%
2 step prediction accuracy: 86%
5 step prediction accuracy: 81%
10 step prediction accuracy: 75%
15 step prediction accuracy: 70%
30 step prediction accuracy: 61%
60 step prediction accuracy: 48%
